In [1]:
import sys
import os
from timeit import default_timer as timer
import torch
from transformers import RobertaTokenizer, RobertaModel, RobertaForMaskedLM, RobertaConfig
import matplotlib.pyplot as plt
# % matplotlib inline
from scipy.spatial.distance import cosine

In [2]:
# Make sure we're in the transformers directory with fine-tuned model output.
os.chdir('/home/jupyter/Notebooks/crystal/NLP/transformers/examples/')
os.getcwd()

'/home/jupyter/Notebooks/crystal/NLP/transformers/examples'

In [3]:
# Adapted from the tutorial at https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
# and Transformers documentation: https://huggingface.co/transformers/model_doc/roberta.html#robertaformaskedlm

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('./roBERTa_base')
config = RobertaConfig.from_pretrained('./roBERTa_base')
model = RobertaForMaskedLM.from_pretrained('./roBERTa_base')
model.eval()

context_file = "/home/jupyter/Notebooks/crystal/NLP/transformers/examples/CC_WET_train_aefg"
output_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/'
count_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/'
vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/data/vocab_files/SimLex_MC_MiFace_vocab.txt'
vocab = make_vocab(vocab_file)

FEATURE_COUNT = 768
LAYER = 8
MAX_LINES = 50

In [12]:
l_output_file = ''
l_count_file = ''
l_output_file = os.path.join(output_file, 'roberta_base_layer-' + str(LAYER) + '_maxlines-' + str(MAX_LINES) + '_aefg.txt')
l_count_file = os.path.join(count_file, 'roberta_base_layer-'+ str(LAYER) + '_maxlines-' + str(MAX_LINES) + '_aefg_counts.txt')
print(l_output_file, l_count_file)
# Process vocabulary words in the outer loop.
for v in vocab:
    start = timer()
    with open(context_file, 'r') as lines:
        v_sum = torch.zeros([1, FEATURE_COUNT])
        v_tokens = tokenizer.encode(v)
        print(f'\nThere are {len(v_tokens) - 2} tokens in tokenized vocabulary word:')
        for t in v_tokens[1:-1]:
            print(tokenizer.decode(t).strip())
        count_sentence = 0
        count_tensor = 0

        # Process all lines in the context file in the inner loop.
        for line in lines:
            # Check for this vocab word in this line; if found, split the line into individual sentences.
            if v in line.lower().split():
                for sentence in line.split('.'):
                    if v in sentence.lower():
                        line = sentence
                        count_sentence += 1
                        break
                # Split the new sentence-based line into tokens.
                # Use max_length to avoid overflowing the maximum sequence length for the model.
                tokenized_text = tokenizer.encode(line, add_special_tokens=True, max_length=512)
                indices = []              

                # Check to see whether the vocab word is found in this particular line.
                # Initially, some lines may have comprised multiple sentences, which were
                # broken out individually above.
                for t in v_tokens[1:-1]:
                    for i, token_str in enumerate(tokenized_text):
                        if tokenizer.decode(token_str).strip() == tokenizer.decode(t).strip():
                            indices.append(i)               

                ###################################################################################
                # If the vocabulary word was found, process the containing line.
                if indices:

                    # The vocab word was found in this line/sentence, at the locations in indices.
                    # Get the feature vectors for all tokens in the line/sentence.
                    token_embeddings = create_token_embeddings(tokenized_text)
                    token_vecs_layer = get_layer_token_vecs(token_embeddings, LAYER)

                    # Get the vocab word's contextual embedding for this line.
                    tensor_layer = torch.zeros([1, FEATURE_COUNT])
                    for i in range(len(indices)):
                        v_index = i % len(v_tokens[1:-1])
                        tensor_layer += token_vecs_layer[indices[i]]

                    # If our vocab word is broken into more than one token, we need to get the mean of the token embeddings.
                    tensor_layer /= len(indices)

                    # Add the embedding distilled from this line to the sum of embeddings for all lines.
                    v_sum += tensor_layer
                    count_tensor += 1
                ###################################################################################
            # Stop processing lines once we've found 2000 instances of our vocab word.
            if count_tensor >= MAX_LINES:
                break

        # We're done processing all lines of 512 tokens or less containing our vocab word.
        # Get the mean embedding for the word.
        v_mean = v_sum / count_tensor
        print(f'Mean of {count_tensor} tensors is: {v_mean[0][:5]} ({len(v_mean[0])} features in tensor)')
        write_embedding(l_output_file, v, v_mean)
        try:
            with open(l_count_file, 'a') as counts:
                counts.write(v + ', ' + str(count_tensor) + '\n')
        except:
            print('Wha?! Could not write the sentence count.')
    end = timer()
    print(f'Run time for {v} was {end - start} seconds.')

/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_base_layer-8_maxlines-50_aefg.txt /home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_base_layer-8_maxlines-50_aefg_counts.txt

There are 1 tokens in tokenized vocabulary word:
aback
Mean of 50 tensors is: tensor([0.0678, 0.2718, 0.1138, 0.4885, 0.0466]) (768 features in tensor)
Run time for aback was 14.45582143124193 seconds.

There are 2 tokens in tokenized vocabulary word:
ab
ashed
Mean of 24 tensors is: tensor([ 0.1307,  0.3149, -0.0467,  0.1518,  0.4097]) (768 features in tensor)
Run time for abashed was 102.83370731212199 seconds.

There are 1 tokens in tokenized vocabulary word:
abdomen
Mean of 50 tensors is: tensor([ 0.4638, -0.1044,  0.0831,  0.3742, -1.0691]) (768 features in tensor)
Run time for abdomen was 9.522613850422204 seconds.

There are 1 tokens in tokenized vocabulary word:
abhor
Mean of 50 tensors is: tensor([0.1734, 0.3307, 0.1492, 0.3795, 0.1214]) (768 f

Mean of 50 tensors is: tensor([ 0.1961, -0.2198,  0.0692, -0.6464, -0.5064]) (768 features in tensor)
Run time for actress was 10.233919771388173 seconds.

There are 1 tokens in tokenized vocabulary word:
acute
Mean of 50 tensors is: tensor([ 0.1484,  0.1825, -0.2688,  0.3861,  0.5295]) (768 features in tensor)
Run time for acute was 4.487680409103632 seconds.

There are 1 tokens in tokenized vocabulary word:
adamant
Mean of 50 tensors is: tensor([-0.0148,  0.3897, -0.0857, -0.2592, -0.3916]) (768 features in tensor)
Run time for adamant was 9.49912191182375 seconds.

There are 1 tokens in tokenized vocabulary word:
add
Mean of 50 tensors is: tensor([ 0.3383,  0.5262,  0.1888, -0.1093, -1.2622]) (768 features in tensor)
Run time for add was 3.2420229418203235 seconds.

There are 2 tokens in tokenized vocabulary word:
add
led
Mean of 50 tensors is: tensor([0.1653, 0.3888, 0.0771, 0.2757, 0.0293]) (768 features in tensor)
Run time for addled was 40.88772522006184 seconds.

There are 1 to

Mean of 50 tensors is: tensor([ 0.3920, -0.0249,  0.0183, -0.9810, -0.3431]) (768 features in tensor)
Run time for air was 3.8291665110737085 seconds.

There are 2 tokens in tokenized vocabulary word:
air
head
Mean of 50 tensors is: tensor([ 0.1502, -0.1447, -0.0410, -0.6845,  0.1570]) (768 features in tensor)
Run time for airhead was 54.85662148613483 seconds.

There are 1 tokens in tokenized vocabulary word:
airport
Mean of 50 tensors is: tensor([-0.0867,  0.8159,  0.0223, -0.7761, -0.6384]) (768 features in tensor)
Run time for airport was 4.5908777276054025 seconds.

There are 1 tokens in tokenized vocabulary word:
aisle
Mean of 50 tensors is: tensor([ 0.1025,  0.5465, -0.0319, -0.4098, -0.8816]) (768 features in tensor)
Run time for aisle was 7.818837974220514 seconds.

There are 1 tokens in tokenized vocabulary word:
alarm
Mean of 50 tensors is: tensor([ 0.1356,  0.2249, -0.1112, -0.5541, -0.3347]) (768 features in tensor)
Run time for alarm was 4.900718811899424 seconds.

There 

Mean of 50 tensors is: tensor([0.1501, 0.2755, 0.2214, 0.2111, 0.1267]) (768 features in tensor)
Run time for angered was 12.460219966247678 seconds.

There are 1 tokens in tokenized vocabulary word:
angle
Mean of 50 tensors is: tensor([ 0.1606,  0.7239,  0.1291,  0.5848, -0.1953]) (768 features in tensor)
Run time for angle was 5.274487386457622 seconds.

There are 1 tokens in tokenized vocabulary word:
angrily
Mean of 50 tensors is: tensor([ 0.4645,  0.3924, -0.1596,  0.0620,  0.2278]) (768 features in tensor)
Run time for angrily was 14.52509154099971 seconds.

There are 1 tokens in tokenized vocabulary word:
angry
Mean of 50 tensors is: tensor([0.0321, 0.1823, 0.0149, 0.6952, 0.1702]) (768 features in tensor)
Run time for angry was 4.628210744820535 seconds.

There are 1 tokens in tokenized vocabulary word:
angst
Mean of 50 tensors is: tensor([ 0.2315,  0.1470,  0.0293, -0.0009, -0.0442]) (768 features in tensor)
Run time for angst was 9.81400201190263 seconds.

There are 1 tokens 

Mean of 50 tensors is: tensor([ 0.2321,  0.3407, -0.0770, -0.8135, -0.8890]) (768 features in tensor)
Run time for appliance was 10.992344048805535 seconds.

There are 1 tokens in tokenized vocabulary word:
appoint
Mean of 50 tensors is: tensor([-0.0145,  0.2525,  0.0441,  0.1105, -1.5270]) (768 features in tensor)
Run time for appoint was 8.04832282755524 seconds.

There are 1 tokens in tokenized vocabulary word:
appointment
Mean of 50 tensors is: tensor([ 0.2094,  0.4981,  0.2017,  0.0540, -0.6496]) (768 features in tensor)
Run time for appointment was 4.962004272267222 seconds.

There are 2 tokens in tokenized vocabulary word:
appreci
ative
Mean of 50 tensors is: tensor([-0.0136,  0.2793, -0.0094,  0.7884, -1.0356]) (768 features in tensor)
Run time for appreciative was 5.797259575687349 seconds.

There are 1 tokens in tokenized vocabulary word:
apprehension
Mean of 50 tensors is: tensor([ 0.1369,  0.0151,  0.0641,  0.0508, -0.7101]) (768 features in tensor)
Run time for apprehensio

...

Mean of 50 tensors is: tensor([-0.0319,  0.3065,  0.2233, -0.1792, -0.3809]) (768 features in tensor)
Run time for unsuspecting was 8.170786353759468 seconds.

There are 3 tokens in tokenized vocabulary word:
uns
way
ed
Mean of 7 tensors is: tensor([-0.0750,  0.2926,  0.0680, -0.0693, -0.2748]) (768 features in tensor)
Run time for unswayed was 107.37240052036941 seconds.

There are 4 tokens in tokenized vocabulary word:
un
sy
mp
athetic
Mean of 50 tensors is: tensor([ 0.1180,  0.4604,  0.0340, -0.1777, -0.3811]) (768 features in tensor)
Run time for unsympathetic was 20.451607468537986 seconds.

There are 1 tokens in tokenized vocabulary word:
untouched
Mean of 50 tensors is: tensor([ 0.1229,  0.0123,  0.0778, -0.3082, -0.5517]) (768 features in tensor)
Run time for untouched was 7.1679805014282465 seconds.

There are 3 tokens in tokenized vocabulary word:
unt
rou
bled
Mean of 50 tensors is: tensor([ 0.1530,  0.2844,  0.1118,  0.9100, -0.1374]) (768 features in tensor)
Run time for un

Mean of 50 tensors is: tensor([ 0.2368,  0.4744,  0.0737, -0.1311, -1.0963]) (768 features in tensor)
Run time for victory was 8.255805813707411 seconds.

There are 1 tokens in tokenized vocabulary word:
vigilant
Mean of 50 tensors is: tensor([0.0120, 0.0240, 0.1904, 0.4588, 0.1117]) (768 features in tensor)
Run time for vigilant was 8.708358452655375 seconds.

There are 1 tokens in tokenized vocabulary word:
vile
Mean of 50 tensors is: tensor([ 0.1226, -0.0963, -0.3048, -0.1104,  0.3570]) (768 features in tensor)
Run time for vile was 7.957913567312062 seconds.

There are 2 tokens in tokenized vocabulary word:
villain
ous
Mean of 50 tensors is: tensor([ 0.0360,  0.3529, -0.0610,  0.5068,  0.5855]) (768 features in tensor)
Run time for villainous was 15.106521492823958 seconds.

There are 2 tokens in tokenized vocabulary word:
vind
ictive
Mean of 50 tensors is: tensor([ 0.2095,  0.4280,  0.0255,  0.5463, -0.1792]) (768 features in tensor)
Run time for vindictive was 16.90362366102636 s

Mean of 50 tensors is: tensor([ 0.3081, -0.3787,  0.2929, -1.0552, -0.2114]) (768 features in tensor)
Run time for water was 3.9721388472244143 seconds.

There are 3 tokens in tokenized vocabulary word:
wa
ver
ing
Mean of 50 tensors is: tensor([ 0.0835,  0.3740, -0.4752,  0.0160, -0.1899]) (768 features in tensor)
Run time for wavering was 28.041507499292493 seconds.

There are 1 tokens in tokenized vocabulary word:
way
Mean of 50 tensors is: tensor([ 0.3791,  0.7630, -0.2124,  0.1575, -0.4916]) (768 features in tensor)
Run time for way was 3.9884781166911125 seconds.

There are 1 tokens in tokenized vocabulary word:
wealth
Mean of 50 tensors is: tensor([ 0.1325,  0.2591,  0.2326, -0.8832,  0.0700]) (768 features in tensor)
Run time for wealth was 4.4340178379788995 seconds.

There are 2 tokens in tokenized vocabulary word:
wear
iness
Mean of 50 tensors is: tensor([ 0.0666,  0.2888,  0.2462, -0.0723,  0.1059]) (768 features in tensor)
Run time for weariness was 31.870356270112097 secon

Mean of 50 tensors is: tensor([ 0.2103,  0.4982, -0.0742, -0.0582,  0.0650]) (768 features in tensor)
Run time for wistfully was 31.265798576176167 seconds.

There are 1 tokens in tokenized vocabulary word:
withdraw
Mean of 50 tensors is: tensor([ 0.1159,  0.3406,  0.0944,  0.2547, -0.6918]) (768 features in tensor)
Run time for withdraw was 4.274333788082004 seconds.

There are 1 tokens in tokenized vocabulary word:
withdrawn
Mean of 50 tensors is: tensor([ 0.0320,  0.1449,  0.1124,  0.2429, -0.1145]) (768 features in tensor)
Run time for withdrawn was 5.612368171103299 seconds.

There are 1 tokens in tokenized vocabulary word:
withheld
Mean of 50 tensors is: tensor([ 0.1312,  0.0270, -0.2623, -0.2566, -0.2409]) (768 features in tensor)
Run time for withheld was 7.029346066527069 seconds.

There are 1 tokens in tokenized vocabulary word:
withholding
Mean of 50 tensors is: tensor([ 0.1257,  0.0233, -0.2398, -0.2796,  0.0730]) (768 features in tensor)
Run time for withholding was 7.0876

Mean of 50 tensors is: tensor([ 0.1381, -0.0495, -0.2294, -0.7276,  0.3529]) (768 features in tensor)
Run time for yawning was 15.613852648064494 seconds.

There are 1 tokens in tokenized vocabulary word:
year
Mean of 50 tensors is: tensor([-0.0807,  0.4071,  0.0468, -0.2169, -0.4440]) (768 features in tensor)
Run time for year was 3.5877525759860873 seconds.

There are 2 tokens in tokenized vocabulary word:
year
ning
Mean of 50 tensors is: tensor([ 0.1150, -0.1523,  0.0150,  0.2728,  0.1582]) (768 features in tensor)
Run time for yearning was 7.917504573240876 seconds.

There are 1 tokens in tokenized vocabulary word:
yell
Mean of 50 tensors is: tensor([ 0.2217, -0.1585, -0.2781,  0.0770, -0.4935]) (768 features in tensor)
Run time for yell was 7.113133622333407 seconds.

There are 1 tokens in tokenized vocabulary word:
yelling
Mean of 50 tensors is: tensor([ 0.0806, -0.3010, -0.4002,  0.4312, -0.0537]) (768 features in tensor)
Run time for yelling was 6.7533686356619 seconds.

There 

In [5]:
def make_vocab(vocab_file):
    vocab = []
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    return vocab

In [6]:
def create_token_embeddings(tokenized_text):
    input_ids = torch.tensor(tokenized_text).unsqueeze(0)  # Batch size 1
    with torch.no_grad():
        outputs = model(input_ids, masked_lm_labels=input_ids)
        encoded_layers = outputs[2]
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
#         print(f'Size of token embeddings is {token_embeddings.size()}')
        return token_embeddings

In [7]:
# Sum the last 4 layers' features
def sum_last_four_token_vecs(token_embeddings):
    token_vecs_sum_last_four = []

    # For each token in the sentence...
    for token in token_embeddings:
        # `token` is a [13 x 768] tensor
        # Sum the vectors from the last 4 layers.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum_last_four.append(sum_vec)

#     print ('Shape of summed layers is: %d x %d' % (len(token_vecs_sum_last_four), len(token_vecs_sum_last_four[0])))
    # Shape is: <token count> x 768
    return token_vecs_sum_last_four

In [8]:
# Return a single layer of the model.
def get_layer_token_vecs(token_embeddings, layer_number):
    token_vecs_layer = []

    # For each token in the sentence...
    for token in token_embeddings:
        # `token` is a [13 x 768] tensor
        # Sum the vectors from the last 4 layers.
        layer_vec = token[layer_number]

        # Use `sum_vec` to represent `token`.
        token_vecs_layer.append(layer_vec)

#     print ('Shape of summed layers is: %d x %d' % (len(token_vecs_layer), len(token_vecs_layer[0])))
    # Shape is: <token count> x 768
    return token_vecs_layer

In [9]:
def write_embedding(embeddings_file, vocab_word, contextual_embedding):
    try:
        with open(embeddings_file, 'a') as f:
            f.write(vocab_word)
            for value in contextual_embedding[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
#         print(f'Saved the embedding for {vocab_word}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')